In [ ]:
!pip install pandas tqdm

In [ ]:
import ast
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
omop_ancestors = pd.read_csv('CONCEPT_ANCESTOR.csv', sep='\t') # Available at OHDSI Athena (athena.ohdsi.org)

In [ ]:
results_df = pd.read_csv('results.csv', sep='\t', header=0) # Output CSV from RAG/agent

In [ ]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else None
    except (ValueError, SyntaxError):
        return []

In [ ]:
results_df['Annotations'] = results_df['Annotations'].progress_apply(safe_literal_eval)
results_df['llm_concept'] = results_df['llm_concept'].progress_apply(safe_literal_eval)
results_df['llm_name'] = results_df['llm_name'].progress_apply(safe_literal_eval)

In [ ]:
def check_hierarchy(row):
    if row.llm_concept and not row.partial_match and not row.exact_match:
        for annotation in row.Annotations:
            annotated_concept = annotation.get('conceptId')
            llm_concept = row.llm_concept[0]
            ancestors = omop_ancestors[(omop_ancestors.ancestor_concept_id == int(annotated_concept)) & (omop_ancestors.descendant_concept_id == int(llm_concept))]
            descendants = omop_ancestors[(omop_ancestors.ancestor_concept_id == int(llm_concept)) & (omop_ancestors.descendant_concept_id == int(annotated_concept))]
            if not ancestors.empty:
                return True
            if not descendants.empty:
                return True

In [ ]:
results_df['hierarchy_match'] = results_df.progress_apply(check_hierarchy, axis=1)

In [ ]:
print('Number of hierarchy matches (LLM candidates) = ', len(results_df[results_df.hierarchy_match == True]))